In [53]:
import schematics

{
    "metadata": {
        "name": "Johan",
        "displayName": "Johan Anderson",
        "description": "System admin",
    },
    "userDef": {
        "firstName": "Johan",
        "lastName": "Anderson",
        "email": "johan@comix.com",
    }
}

def validate_no_space(value):
    """fields must not have spaces."""
    if " " in value:
        raise schematics.exceptions.ValidationError("There must not be spaces in value")


class UserMetadata(schematics.Model):
    name = schematics.types.StringType(required=True)
    display_name = schematics.types.StringType(serialized_name="displayName")
    description = schematics.types.StringType()


class UserDef(schematics.Model):
    first_name = schematics.types.StringType(serialized_name="firstName", validators=[validate_no_space])
    last_name = schematics.types.StringType(serialized_name="lastName", validators=[validate_no_space])
    email = schematics.types.EmailType()


class User(schematics.Model):
    metadata = schematics.types.ModelType(UserMetadata)
    user_def = schematics.types.ModelType(UserDef, serialized_name="userDef")

In [35]:
ud = UserDef({
        "firstName": "Johan",
        "lastName": "Anderson",
        "email": "johan@comix.com",
})

In [36]:
ud.first_name

'Johan'

In [4]:
ud.last_name

'Anderson'

In [5]:
ud.email

'johan@comix.com'

In [8]:
um = UserMetadata({
        "name": "Johan",
        "displayName": "Johan Anderson",
        "description": "System admin",
})

In [9]:
um.name

'Johan'

In [10]:
um.display_name

'Johan Anderson'

In [11]:
um.description

'System admin'

In [18]:
u = User({
    "metadata": {
        "name": "Johan",
        "displayName": "Johan Anderson",
        "description": "System admin",
    },
    "userDef": {
        "firstName": "Johan",
        "lastName": "Anderson",
        "email": "johan@comix.com",
    }
})

In [19]:
u.metadata

<UserMetadata instance>

In [20]:
u.metadata.name

'Johan'

In [21]:
u.user_def.email

'johan@comix.com'

# Validate UserDef

In [55]:
# Should be OK
ud = UserDef({"email": "foo@bar.com"})
ud.validate()

In [54]:
ud = UserDef(dict(email="ax", firstName="Boo Ba", lastName="Blah blah"))
try:
    ud.validate()
except schematics.exceptions.DataError as error:
    print("The following faults found:")
    for field_name, faults in error.to_primitive().items():
        print(f"- {field_name}:")
        for fault in faults:
            print(f"   - {fault}")
    print("---")
    print(error)

The following faults found:
- firstName:
   - There must not be spaces in value
- lastName:
   - There must not be spaces in value
- email:
   - Not a well-formed email address.
---
{"firstName": ["There must not be spaces in value"], "lastName": ["There must not be spaces in value"], "email": ["Not a well-formed email address."]}


# Validate User

In [57]:
# Should be OK
u = User({
    "metadata": {
        "name": "Johan",
        "displayName": "Johan Anderson",
        "description": "System admin",
    },
    "userDef": {
        "firstName": "Johan",
        "lastName": "Anderson",
        "email": "johan@comix.com",
    }
})
u.validate()

In [58]:
# Email and name errors
u = User({
    "metadata": {
        "name": "Johan",
        "displayName": "Johan Anderson",
        "description": "System admin",
    },
    "userDef": {
        "firstName": "Johan Ken",
        "lastName": "Anderson",
        "email": "johan",
    }
})

try:
    u.validate()
except schematics.exceptions.DataError as error:
    print(error)

{"userDef": {"firstName": ["There must not be spaces in value"], "email": ["Not a well-formed email address."]}}


In [60]:
# name is missing
u = User({
    "metadata": {
        "displayName": "Johan Anderson",
        "description": "System admin",
    },
    "userDef": {
        "firstName": "Johan",
        "lastName": "Anderson",
        "email": "johan@comix.com",
    }
})

try:
    u.validate()
except schematics.exceptions.DataError as error:
    print(error)

{"metadata": {"name": ["This field is required."]}}
